In [47]:
import pandas as pd
import numpy as np

import psycopg2

from dotenv import load_dotenv
import os

load_dotenv()

True

In [2]:
DATA_URL = "https://archive.ics.uci.edu/ml/machine-learning-databases/00502/online_retail_II.xlsx"

# with pd.ExcelFile(
#         "https://archive.ics.uci.edu/ml/machine-learning-databases/00502/online_retail_II.xlsx"
# ) as xls:
#     df1 = pd.read_excel(xls)

# print(df1)

raw_data = pd.read_excel(DATA_URL, sheet_name=None)
print(raw_data)
print(raw_data.keys())

{'Year 2009-2010':        Invoice StockCode                          Description  Quantity  \
0       489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1       489434    79323P                   PINK CHERRY LIGHTS        12   
2       489434    79323W                  WHITE CHERRY LIGHTS        12   
3       489434     22041         RECORD FRAME 7" SINGLE SIZE         48   
4       489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   
...        ...       ...                                  ...       ...   
525456  538171     22271                 FELTCRAFT DOLL ROSIE         2   
525457  538171     22750         FELTCRAFT PRINCESS LOLA DOLL         1   
525458  538171     22751       FELTCRAFT PRINCESS OLIVIA DOLL         1   
525459  538171     20970   PINK FLORAL FELTCRAFT SHOULDER BAG         2   
525460  538171     21931               JUMBO STORAGE BAG SUKI         2   

               InvoiceDate  Price  Customer ID         Country  
0      2009-12-

In [3]:
df = raw_data['Year 2009-2010']

df.columns = ['invoice_id', 'product_id', 'description', 'quantity', 'datetime', 'price', 'customer_id', 'country']

df.head()

,invoice_id,product_id,description,quantity,datetime,price,customer_id,country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [4]:
df.isnull().sum()

# print(df.shape)

# # very simple N/A removal, doesn't do anything
# df = df.dropna(subset=['Invoice', 'StockCode', 'Customer ID'])

# print(df.shape)

# df.head()

invoice_id          0
product_id          0
description      2928
quantity            0
datetime            0
price               0
customer_id    107927
country             0
dtype: int64

In [5]:
import uuid

df['transaction_id'] = [uuid.uuid4().int for i in range(len(df))]

df = df.sort_values('datetime', ascending=True)


In [63]:
def join_uuid(df, column_name, new_column_name = 'id'):

    unique_items = df[column_name].unique()

    invoice_id_key = pd.DataFrame({
        new_column_name: [uuid.uuid4().int for i in range(len(unique_items))],
        column_name: unique_items
    })

    return df.merge(invoice_id_key, on=column_name)


In [7]:
df['datetime'] = pd.to_datetime(df['datetime'])

df = join_uuid(df, 'datetime', 'date_id')


df.head()

,invoice_id,product_id,description,quantity,datetime,price,customer_id,country,transaction_id,date_id
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,177298128883685068600431429334522346233,133823699918563960823878320872831392311
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,22880744713223009797236749495925253608,133823699918563960823878320872831392311
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,22077985645692962446140876774181330150,133823699918563960823878320872831392311
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,98716072285114842041317620186194984546,133823699918563960823878320872831392311
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,583699593579771258422401027596383920,133823699918563960823878320872831392311


In [8]:
# Fact Table

# transactions

df_fact = df[['transaction_id', 'invoice_id', 'product_id', 'customer_id', 'quantity', 'price']]

df_fact.head()

,transaction_id,invoice_id,product_id,customer_id,quantity,price
0,177298128883685068600431429334522346233,489434,85048,13085.0,12,6.95
1,22880744713223009797236749495925253608,489434,79323P,13085.0,12,6.75
2,22077985645692962446140876774181330150,489434,79323W,13085.0,12,6.75
3,98716072285114842041317620186194984546,489434,22041,13085.0,48,2.10
4,583699593579771258422401027596383920,489434,21232,13085.0,24,1.25


In [14]:
# Dimension Tables

# Invoice table

invoices = df.groupby('invoice_id').agg({
    'date_id': 'first'
}).reset_index()

invoices['cancelled'] = invoices['invoice_id'].str.startswith('C', na=False)


In [10]:

## Date Table

dates = df.groupby('date_id').agg({'datetime': 'first'})

# Create other date features

dates['year'] = dates['datetime'].dt.year
dates['quarter'] = dates['datetime'].dt.quarter
dates['month'] = dates['datetime'].dt.month
dates['day'] = dates['datetime'].dt.day_name()
dates['hour'] = dates['datetime'].dt.hour

dates.head()


,datetime,year,quarter,month,day,hour
date_id,,,,,,
16602478707615772848357651424154260,2010-07-22 15:33:00,2010,3,7,Thursday,15
26758125834187263988116624931873141,2010-12-02 17:09:00,2010,4,12,Thursday,17
28444483104696168388447513012178030,2010-03-24 16:22:00,2010,1,3,Wednesday,16
60260244185884178136070333308509692,2010-09-22 11:55:00,2010,3,9,Wednesday,11
69982606605839233732556871667386367,2010-09-19 15:36:00,2010,3,9,Sunday,15


In [11]:
## Product Table
# most recent price and description
products = df.groupby('product_id').agg({
        'description': 'first', 
        'price': 'first'
    })



,description,price
product_id,,
10002,INFLATABLE POLITICAL GLOBE,0.85
10080,GROOVY CACTUS INFLATABLE,0.85
10109,BENDY COLOUR PENCILS,0.42
10120,DOGGY RUBBER,0.21
10125,MINI FUNKY DESIGN TAPES,1.70


In [64]:
## Customer Table

df = join_uuid(df, 'country', 'country_id')


customers = df.groupby('customer_id').agg({
                               'country_id': 'first'  # most recent country
                           }).reset_index()


In [101]:

## Country Table

countries = df.groupby('country_id').agg({
                               'country':'first'
                           }).reset_index()

countries.head()


,country_id,country
0,3695474070791635129583157635796926703,United Kingdom
1,4112150696745488679617005825264533782,Lithuania
2,4172645294300822428271636474857681939,Spain
3,11012710800552373886484978878424781062,Portugal
4,17157234891645266115985710938399692312,Poland


In [102]:
import requests


def get_latlon(country_name):

    url = "https://nominatim.openstreetmap.org/search"

    params = {
        "country": country_name,
        "format": "json"
    }

    response = requests.get(url, params=params).json()

    if response:
        return float(response[0].get('lat', np.nan)), float(response[0].get(
            'lon', np.nan))
    else:
        return (np.nan, np.nan)


In [103]:
import math


def great_circle_distance(lat1, lon1, lat2, lon2):
    # convert decimal degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # calculate great-circle distance
    d = math.acos(
        math.sin(lat1) * math.sin(lat2) +
        math.cos(lat1) * math.cos(lat2) * math.cos(lon2 - lon1)) * 6371

    return abs(d)


In [112]:

def get_weather(lat, lon, variable='temperature_2m_max'):

    url = "https://archive-api.open-meteo.com/v1/era5"

    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": "2009-12-01",  # YYYY-MM-DD from dataset info
        "end_date": "2011-12-09",
        "timezone": "UTC",
        "daily": variable
    }

    response = requests.get(url, params=params).json().get('daily', {}).get(variable, [])

    if response:
        return sum(response) / len(response)
    else:
        return np.nan

In [113]:
from tqdm import tqdm

uk_coords = (54.7023545, -3.2765753)

country_stats = []

for country in tqdm(countries['country'].unique()):
    lat, lon = get_latlon(country)
    avg_max_temp = get_weather(lat, lon)
    dist_from_uk = great_circle_distance(lat, uk_coords[0], lon, uk_coords[1])
    country_stats.append((country, lat, lon, avg_max_temp, dist_from_uk))


countries = countries.merge(pd.DataFrame(country_stats,
                                         columns=['country', 'lat', 'lon', 'avg_max_temp', 'dist_from_uk']),
                            on='country')

countries.head()


100%|██████████| 40/40 [01:44<00:00,  2.62s/it]


,country_id,country,lat,lon,avg_max_temp,dist_from_uk
0,3695474070791635129583157635796926703,United Kingdom,54.702354,-3.276575,10.667118,8336.901915
1,4112150696745488679617005825264533782,Lithuania,55.350000,23.750000,10.087009,5849.830730
2,4172645294300822428271636474857681939,Spain,39.326068,-4.837979,21.073478,7693.700632
3,11012710800552373886484978878424781062,Portugal,39.662165,-8.135352,19.948850,7974.276521
4,17157234891645266115985710938399692312,Poland,52.215933,19.134422,11.703924,6174.651808


In [ ]:
countries.head()